In [ ]:
# I am using tensorflow instead of pytorch because my familiarity is more with tensorflow
# All the cells are currently running on google colab

First mount the drive in order to load the dataset from drive

In [131]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now import all the requirements

In [158]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt
import numpy as np
from numpy import load

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
#from keras.preprocessing import image

Firstly we will preprocess the data and make it requirement matching with model. First resize all images (224, 224) and store into a numpy file along with thier labels to make it usable. This code will create a npz file which will contain all the images along with corresponding label.

In [150]:
from os import listdir
from os.path import isdir
from PIL import Image
from numpy import savez_compressed
from numpy import asarray

def pre_process(filename, required_size=(224, 224)):
	# load image from file
	image = Image.open(filename)
	# convert to RGB, if needed
	image = image.convert('RGB')
	# convert to array
	pixels = asarray(image)
	# resize pixels to the model size
	image = Image.fromarray(pixels)
	image = image.resize(required_size)
	img_array = asarray(image)
	return img_array

# load images and resize for all images in a directory
def load_image(directory):
	resized_list = list()
	# enumerate files
	for filename in listdir(directory):
		# path
		path = directory + filename
		# get pre-preprocessed image
		resized_image = pre_process(path)
		# store
		resized_list.append(resized_image)
	return resized_list

# Load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all images in the subdirectory
		processed = pre_process(path)
		# create labels
		labels = [subdir for _ in range(len(processed))]
		# summarize progress
		print('>Loaded %d examples for class: %s' % (len(processed), subdir))
		# store
		X.extend(processed)
		y.extend(labels)
	return asarray(X), asarray(y)

Now give the path of directory which contains all images

In [ ]:
# load train dataset
trainX, trainy = load_dataset('/content/drive/MyDrive/Training Images/DIP_image/train')
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('/content/drive/MyDrive/Training Images/DIP_image/test/')
# save arrays to one file in compressed format
savez_compressed('complete_dataset.npz', trainX, trainy, testX, testy)

Our dataset npz file is ready. Now we need to split into training and testing

In [137]:
# Load the dataset file
data = load('/content/drive/MyDrive/Training Images/complete_dataset.npz')

# Segment it into train and test along with thier label
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded train and test set: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded train and test set:  (150, 224, 224, 3) (150,) (100, 224, 224, 3) (100,)


Now our data is completely ready to feed the model. Now we have to work on model

In [153]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
# print summary to see the output features shape
print(base_model.summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_15[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

Now we need to extract the features from the last layer of model. So we can see in the summary that last layer contains (None, 7, 7, 2048) features in output. KNN need data in point form hence make features appropriate

In [154]:
#Just feed the train and test image
trainX_features = base_model.predict(trainX)
testX_features = base_model.predict(testX)
print("Features extracted")
# Make it matching with KNN
trainX_features = np.reshape(trainX_features, (150, 7*7* 2048))                 # Total 150 images and each image contains 7X7 2048 features
testX_features = np.reshape(testX_features, (100, 7*7*2048))

4/4 [==============================] - 15s 3s/step
Features extracted


In [155]:
# Do labeling of the testy and trainy
for i in range(0,150):
  if (trainy[i]=='butterfly_train'):
    trainy[i]= 'butterfly'
  if (trainy[i]=='dog_train'):
    trainy[i]='dog'
  if (trainy[i]=='mountains_train'):
    trainy[i]='mountains'
  if (trainy[i]=='bird_train'):
    trainy[i]='bird'
  if (trainy[i]=='Fish_train'):
    trainy[i]= 'Fish'
# And also for test data  
for i in range(0,100):
  if (testy[i]=='butterfly_test'):
    testy[i]= 'butterfly'
  if (testy[i]=='dog_test'):
    testy[i]= 'dog'
  if (testy[i]=='mountains_test'):
    testy[i]= 'mountains'
  if (testy[i]=='bird_test'):
    testy[i]= 'bird'
  if (testy[i]=='Fish_test'):
    testy[i]= 'Fish'
  


Now make fit the features in KNN and predict 

In [156]:
KNN_Model = KNeighborsClassifier(n_neighbors=3)
KNN_Model.fit(trainX_features, trainy)

predict_output = KNN_Model.predict(testX_features)

And now print the accuracy of the model

In [157]:
print('Accuracy of the model is :', accuracy_score(predict_output, testy))

Accuracy of the model is : 0.86
